![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx.ai and LangChain Agents to perform sequence of actions

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate usage of LangChain Agents with watsonx.ai models.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The purpose of this notebook is to demonstrate how to use language models, e.g. `meta-llama/llama-3-70b-instruct`, to create Structured Chat LangChain Agent with memory, where an Agent may perform sequence of actions based on the model responses and assures that LLM uses chat memory to provide the most accurate answer on the user question.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [LangChain integration](#watsonxllm)
- [Prompt template and custom tools](#prompt_template)
- [LangChain Agent](#agent)
- [Example of Structured Chat](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependecies

In [ ]:
!pip install "ibm-watsonx-ai>=1.0.1" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install "langchain==0.2.1" | tail -n 1
!pip install "langchain-ibm==0.1.7" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [1]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="watsonxllm"></a>
## Initialize `WatsonxLLM` instance

#### Set model parameters

To set the model parameters we use `GenTextParamsMetaNames` metanames from `ibm-watsonx-ai`.

In [3]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
from langchain_ibm import WatsonxLLM

Based on the performed experiments we found that the following parameters work well with the LangChain Agents created in the further section.

In [4]:
parameters = {
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 200,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 10,
    GenTextParamsMetaNames.DECODING_METHOD: 'greedy',
}

Furthermore, you need to specify `model_id` that will be used for inferencing. In the following notebook we use `meta-llama/llama-2-70b-chat`.

In [6]:
llm_chat = WatsonxLLM(
    url=credentials.get('url'),
    apikey=credentials.get('apikey'),
    project_id=project_id,
    model_id="meta-llama/llama-3-70b-instruct",
    params=parameters
)

In [7]:
response = llm_chat.invoke('What is a generative AI? Respond concisely.')
print(response)

**

A generative AI is a type of artificial intelligence that creates new, original content, such as images, videos, music, text, or code, using machine learning algorithms and statistical models.


<a id="prompt_template"></a>
## Prompt template preparation and tools
### Create Prompt Template

A structured chat prompt template is composed of several sections: system prompt, chat history and user input. The following prompt template was prepared based on the LangChain [documentation](https://python.langchain.com/docs/modules/agents/agent_types/structured_chat) and adjusted to work with the chat models available on watsonx.ai.

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import MessagesPlaceholder

In [9]:
system_prompt = """Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. 

AI assistant has access to the following tools:

{tools}

Use a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). 

Use only valid "action" values: "Final Answer" or {tool_names} and provide only one action in every response. Use "action" value "Final Answer" if you know what to respond. Follow this format:

Thought: consider previous and subsequent steps
Action:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Observation: action result

Thought: I know what to respond
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```

Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools only if necessary.
"""

In [10]:
user_input = '{input}\n{agent_scratchpad}\n(reminder to respond in a JSON blob no matter what and use tools only if necessary)'

To create the final structured chat prompt template one can use initialization class method from `ChatPromptTemplate` .

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_prompt
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", user_input),
    ]
)

In [12]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Below are a series of dialogues between various people and an AI assistant. The AI tries to be helpful, polite, honest, sophisticated, emotionally aware, and humble-but-knowledgeable. The assistant prioritizes caution over usefulness, refusing to answer questions that it considers unsafe, immoral, unethical or dangerous. \n\nAI assistant has access to the following tools:\n\n{tools}\n\nUse a JSON blob to specify a tool by providing an action key (tool name) and an action input key (tool input). \n\nUse only valid "action" values: "Final Answer" or {tool_names} and provide only one action in every response. Use "action" value "Final Answer" if you know what to respond. Follow this format:\n\nThought: consider previous and subsequent steps\nAction:\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\nObservation: action result\n\nThought: I know what to respond\n```\n{{\

### Prepare tools

In this section we prepare tools (python functions) that the AI assistant will have access to.

In [13]:
from langchain.agents import tool

In [14]:
@tool
def get_word_length(word: str) -> int:
    """Return the length of the given word."""
    return len(word)

@tool
def remove_punctuations(text: str) -> str:
    """Remove all punctuation symbols from the text."""
    import re 

    new_string = re.sub(r'[^\w\s]','',text) 
    return new_string

@tool
def get_mean_from_sequence(sequence: str) -> float:
    """Return the mean value from the sequence of numbers."""
    import numpy
    try:
        return numpy.mean(numpy.array(sequence.split(','), dtype=float))
    except:
        return "Elements of `sequence` cannot be converted to float"

tools_chat = [get_word_length, get_mean_from_sequence, remove_punctuations]

Furthermore, we define custom `CustomJSONAgentOutputParser` because sometimes model may return response with additional tokens before action JSON blob. 

In [15]:
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain_core.agents import AgentAction, AgentFinish

In [16]:
class CustomJSONAgentOutputParser(JSONAgentOutputParser):
    def parse(self, text: str) -> AgentAction | AgentFinish:
        i = text.find("{")

        # Sometimes model may return additional tokens before json blob
        text = text[i:]
        if "```" not in text:
            text = "\n```\n" + text
        return super().parse(text + "\n```\nObservation:")

<a id="agent"></a>
## Initialize an Agent

Following section is the central part of the presented notebook. Here, we construct LangChain Agent and initialise `AgentExecutor` which manage conversation with AI assistant, that has access to the predefined tools, and human.

In [17]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor
from langchain.tools.render import render_text_description
from langchain.agents.format_scratchpad import format_log_to_str

In the prompt template, we include the tools signature and their description.

In [18]:
prompt_chat = prompt.partial(
    tools=render_text_description(list(tools_chat)),
    tool_names=", ".join([t.name for t in tools_chat]),
)

Finally, when we have all components prepared we can define Chat Agent and initialize `AgentExecutor`.

In [19]:
# Setting stop sequence for LLM
llm_with_stop = llm_chat.bind(stop=["}\n"])

agent_chat = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt_chat
    | llm_with_stop
    | CustomJSONAgentOutputParser()
)

agent_executor_chat = AgentExecutor(
    agent=agent_chat, tools=tools_chat, verbose=True, handle_parsing_errors=True
)

Notice that we use `handle_parsing_errors=True` to ensure that when LLM is not able to determine what step to take because its output does not have the correct structure to be handled  by output parser, the user's input will be sent back to the model with the appropriate information to improve the model reponse. Setitng this parameter to True help to keep chat conversation smooth, but may also result in multiple requests with many tokens being sent to the model and back.

<a id="chat"></a>
## Structured Chat with memory

To include chat memory in conversation with AI assistant we are going to use `ChatMessageHistory` that simply store chat history in the prompt and send it with every user question to the LLM. 

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor_chat,
    get_session_history=lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

The `session_id` is needed to persist conversation histories. Note that watsonx.ai does not conversation history. Let us start the chat!

In [22]:
agent_with_chat_history.invoke(
    {"input": "Hi! How are you?"},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you for asking!"
}

```
Observation:

> Finished chain.


{'input': 'Hi! How are you?',
 'chat_history': [],
 'output': "I'm doing well, thank you for asking!"}

In [23]:
agent_with_chat_history.invoke(
    {"input": "What is a median?"},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "Final Answer",
  "action_input": "The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values."
}

```
Observation:

> Finished chain.


{'input': 'What is a median?',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!")],
 'output': 'The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'}

In [24]:
agent_with_chat_history.invoke(
    {"input": "Could you provide some example?"},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "Final Answer",
  "action_input": "Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."
}

```
Observation:

> Finished chain.


{'input': 'Could you provide some example?',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!"),
  HumanMessage(content='What is a median?'),
  AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.')],
 'output': "Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."}

Let us now use one of the provided tools.

In [25]:
agent_with_chat_history.invoke(
    {"input": "Calculate the mean value of the sequence `1, 2, -23, 42`."},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "get_mean_from_sequence",
  "action_input": "1, 2, -23, 42"
}

```
Observation:5.5
```
{
  "action": "Final Answer",
  "action_input": "The mean value of the sequence is 5.5."
}

```
Observation:

> Finished chain.


{'input': 'Calculate the mean value of the sequence `1, 2, -23, 42`.',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!"),
  HumanMessage(content='What is a median?'),
  AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'),
  HumanMessage(content='Could you provide some example?'),
  AIMessage(content="Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5.")],
 'output': 'The mean value of the sequence is 5.5.'}

In [26]:
agent_with_chat_history.invoke(
    {"input": "Remove all punctuation symbols from the following sentence `What, is a. loan?!` and return clear sentence."},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "remove_punctuations",
  "action_input": "What, is a. loan?!"
}

```
Observation:What is a loanCould not parse LLM output: 
```
?
```
Observation:Invalid or incomplete response
```
{
  "action": "Final Answer",
  "action_input": "The sentence without punctuation symbols is: What is a loan"
}

```
Observation:

> Finished chain.


{'input': 'Remove all punctuation symbols from the following sentence `What, is a. loan?!` and return clear sentence.',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!"),
  HumanMessage(content='What is a median?'),
  AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'),
  HumanMessage(content='Could you provide some example?'),
  AIMessage(content="Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."),
  HumanMessage(content='Calculate the mean value of the sequence `1, 2, -23, 42`.'),
  AIMessage(content='The mean value of the sequence is 5.5.')],
 'output': 'The sentence without

In [29]:
agent_with_chat_history.invoke(
    {"input": "What is the length of word `think`? "},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "get_word_length",
  "action_input": "think"
}

```
Observation:5
```
{
  "action": "Final Answer",
  "action_input": "The length of the word 'think' is 5."
}

```
Observation:

> Finished chain.


{'input': 'What is the length of word `think`? ',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!"),
  HumanMessage(content='What is a median?'),
  AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'),
  HumanMessage(content='Could you provide some example?'),
  AIMessage(content="Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."),
  HumanMessage(content='Calculate the mean value of the sequence `1, 2, -23, 42`.'),
  AIMessage(content='The mean value of the sequence is 5.5.'),
  HumanMessage(content='Remove all punctuation symbols from the following sentence `What, is a. loan?!`

In [32]:
agent_with_chat_history.invoke(
    {"input": "What is the average length of words `watsonx` and `IBM`? "},
    config={"configurable": {"session_id": "watsonx"}},
)



> Entering new AgentExecutor chain...

```
{
  "action": "get_word_length",
  "action_input": "watsonx"
}

```
Observation:7
```
{
  "action": "get_word_length",
  "action_input": "IBM"
}

```
Observation:3
```
{
  "action": "get_mean_from_sequence",
  "action_input": "7, 3"
}

```
Observation:5.0
```
{
  "action": "Final Answer",
  "action_input": "The average length of the words 'watsonx' and 'IBM' is 5.0."
}

```
Observation:

> Finished chain.


{'input': 'What is the average length of words `watsonx` and `IBM`? ',
 'chat_history': [HumanMessage(content='Hi! How are you?'),
  AIMessage(content="I'm doing well, thank you for asking!"),
  HumanMessage(content='What is a median?'),
  AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'),
  HumanMessage(content='Could you provide some example?'),
  AIMessage(content="Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."),
  HumanMessage(content='Calculate the mean value of the sequence `1, 2, -23, 42`.'),
  AIMessage(content='The mean value of the sequence is 5.5.'),
  HumanMessage(content='Remove all punctuation symbols from the following sentence

History of the chat is stored locally in `message_history` list.

In [33]:
message_history.messages

[HumanMessage(content='Hi! How are you?'),
 AIMessage(content="I'm doing well, thank you for asking!"),
 HumanMessage(content='What is a median?'),
 AIMessage(content='The median is the middle value in a sequence of numbers when they are arranged in order. If the sequence has an even number of values, the median is the average of the two middle values.'),
 HumanMessage(content='Could you provide some example?'),
 AIMessage(content="Let's say we have a sequence of numbers: 1, 3, 5, 7, 9. To find the median, we need to arrange them in order, which they already are. Since there are an odd number of values, the middle value is the median. In this case, the median is 5."),
 HumanMessage(content='Calculate the mean value of the sequence `1, 2, -23, 42`.'),
 AIMessage(content='The mean value of the sequence is 5.5.'),
 HumanMessage(content='Remove all punctuation symbols from the following sentence `What, is a. loan?!` and return clear sentence.'),
 AIMessage(content='The sentence without pun

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use LangChain Agents using custom llm WastonxLLM.
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors:
 **Mateusz Świtała**, Software Engineer at Watson Machine Learning.

Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.